# Measurement of average time intervals

In [ ]:
import pandas as pd


def load_dataset(path):
    # parse the dataset
    df = pd.read_csv(path)

    ################ distilled from notebook 0 ################
    # check the integrity
    assert df.isna().any().any() == False, 'There is at least one missing value.'
    assert df['Timestamp'].is_monotonic_increasing, 'Timestamp is not sorted.'

    # type-cast
    df['abstime'] = pd.to_datetime(df['Timestamp'], unit='s').round('us')
    df['monotime'] = df['Timestamp'] - df['Timestamp'].min()
    df['aid_int'] = df['Arbitration_ID'].map(lambda x: int(x, 16))
    df['y'] = df['Class'].map({'Normal': 0, 'Attack': 1})

    ################ distilled from notebook 1 ################
    # calculate the stream-wise timedelta
    df['Timedelta'] = df.groupby('Arbitration_ID')['Timestamp'].diff()

    return df


df_benign = load_dataset('0_Preliminary/0_Training/Pre_train_D_0.csv')


In [ ]:
#Average and std computation
df_td = pd.concat([
    df_benign.groupby('Arbitration_ID')['Timedelta'].mean().rename('mean_driving'),
    df_benign.groupby('Arbitration_ID')['Timedelta'].std().rename('std_driving'),
], axis=1)
pd.options.display.max_rows = 100
df_td


In [ ]:
#threshold based on 3 std.
df_td['threshold_low'] = df_td['mean_driving'] - 3 * df_td['std_driving']
df_td['threshold_high'] = df_td['mean_driving'] + 3 * df_td['std_driving']
df_td_threshold = df_td[['threshold_low', 'threshold_high']]
df_td_threshold


# Intrusion detection


In [ ]:
df_intrusion = load_dataset('0_Preliminary/0_Training/Pre_train_D_1.csv')
df_intrusion = df_intrusion.join(df_td_threshold, on='Arbitration_ID')

df_intrusion['y_predicted'] = 0  # Init a column with 0
df_detected = df_intrusion.query('not (threshold_low <= Timedelta <= threshold_high)')
df_intrusion.loc[df_detected.index, 'y_predicted'] = 1

abstime_ceil = df_intrusion['abstime'].dt.ceil('10ms')
y = df_intrusion.groupby(abstime_ceil)['y'].max()
y_predicted = df_intrusion.groupby(abstime_ceil)['y_predicted'].max()


# Evaluation


In [ ]:
#Task 5
# Write a code to calculate the TP, FP, TN, and FN. Then using them to calculate the accuracy, precision and recall. The following output is what you should get if done correctly.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print(accuracy_score(y, y_predicted))
print(precision_score(y, y_predicted))
print(recall_score(y, y_predicted))


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y, y_predicted))
print(classification_report(y, y_predicted, digits=4))


Task 6 is to go back and edit the upper sd, lower sd and ceiling values in the code to improve the IDS performance.